In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd

### Sơ qua dữ liệu dùng cho training:
Tên file: train.json

Bao gồm ~18K item đã được gán nhãn. Mỗi item bao gồm:

* **id**: id của mỗi item
* **question**: câu hỏi
* **text**: đoạn text `liên quan` đến câu hỏi, kết quả từ 1 dummy search tool. Là 1 đoạn Wikipedia article.
* **title**: title của Wikipedia article mà đoạn text được lấy từ.
* **label**: nhãn, 2 loại, `True` hoặc `False`, đoạn text trên `có trả lời cho câu hỏi` hoặc `không trả lời được`.

Như ở dưới thì câu hỏi `Quang Hải giành được chức vô địch U21 quốc gia năm bao nhiêu tuổi` có thể xuất hiện nhiều lần, nhưng đi kèm với đoạn text và label tương ứng khác nhau.

In [7]:
# Read Training Data
train_df = pd.read_json('./Dataset/train.json', orient='records')
print('Training data shape: ',train_df.shape)
train_df.head(5)

Training data shape:  (18108, 5)


,id,question,title,text,label
0,u7-1570446247_1,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Năm 2013 , Nguyễn Quang Hải giành chức vô địch...",True
1,u7-1570446247_2,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Sau chức vô địch U-21 quốc gia 2013 , Nguyễn Q...",True
2,u7-1570446247_0,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),Anh bắt đầu gia nhập lò đào tạo trẻ Hà Nội T&T...,False
3,u7-1570446247_3,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Năm 2011 , Nguyễn Quang Hải mới 14 tuổi được g...",False
4,u7-1570445661_0,Mỗi hiệp bóng đá kéo dài bao lâu,Bóng đá,Một trận đấu bóng đá thông thường có hai hiệp ...,True


### Sơ qua dữ liệu dùng cho training:
Tên file: test.json

Bao gồm các test cases có trong tập public test. Mỗi test case bao gồm:

* **__id__**: test case ID
* **question**: câu hỏi
* **paragraphs**: **các** đoạn văn có được từ 1 dummy search tool. Mỗi đoạn có 1 ID riêng **trong test case** (nghĩa là các đoạn trong các test case khác nhau có thể có cùng id). Các đội tham gia phải tìm ID của các đoạn có thể trả lời cho `câu hỏi`.
* **title**: title của Wikipedia article mà đoạn text được lấy từ.

In [15]:
# Read Test Data
test_df = pd.read_json('./Dataset/test.json', orient='records')
print('Test data shape: ',test_df.shape)
test_df.head(5)

Test data shape:  (501, 4)


,__id__,question,title,paragraphs
0,test_0001,Toà nhà nào cao nhất Việt Nam,The Landmark 81,"[{'id': 'p1', 'text': 'The Landmark 81 là một ..."
1,test_0002,Messi thuận chân nào,Lionel Messi,"[{'id': 'p1', 'text': 'Đôi chân ngắn nhưng mạn..."
2,test_0003,Sao Hoả còn được gọi là gì,Sao Hỏa,"[{'id': 'p1', 'text': 'Sao Hoả còn gọi là : Ho..."
3,test_0004,Hồ Hoàn Kiếm nằm ở đâu,Hồ Hoàn Kiếm,"[{'id': 'p1', 'text': 'Hồ Hoàn Kiếm ( chữ Nôm ..."
4,test_0005,Lionel Messi sinh ra ở đâu,Lionel Messi,"[{'id': 'p1', 'text': 'Lionel Andrés "" Leo "" M..."


Ví dụ với câu hỏi trong testcase 2, `Messi thuận chân nào`?, có các đoạn paragraphs sau:

In [25]:
df =pd.DataFrame(test_df['paragraphs'][1])
df.head()

,id,text
0,p1,Đôi chân ngắn nhưng mạnh mẽ giúp Messi dễ dàng...
1,p2,"Cựu huấn luyện viên Barcelona , Pep Guardiola ..."
2,p3,Ngoài ra anh còn một chân sút phạt chính và ha...
3,p4,"Khi đề cập đến kĩ năng dắt bóng của anh , Mara..."
4,p5,"Ngày 11 tháng 3 năm 2010 , Messi được chọn làm..."


### Tìm hiều sâu hơn: